In [178]:
import argparse
import os
from glob import glob

import numpy as np 
import robosuite as suite 
import time

### pop up viewer

In [179]:
# create original environment
env = suite.make(
    "Lift",
    robots="Sawyer",
    ignore_done=True,
    use_camera_obs=False,
    has_renderer=True,
    has_offscreen_renderer=False,
    control_freq=20,
)

In [180]:
fp='/home/ns/sawyer_sim_real/actions_p3.npy'
actions=np.load(fp)
actions.shape

(59, 7)

In [188]:
# env.reset()
env.render()

In [183]:
vel_gotopos(actions[0], kp=0.5)

In [185]:
gripper_open()

In [ ]:
for action in actions:
    vel_gotopos(action, kp=0.5)

In [189]:
env.close()

In [ ]:
# 

In [3]:
fn='/home/ns/wreg/matlab/sawyer_experiments/jps.csv'
fn='/home/ns/wreg/matlab/sawyer_experiments/jps2.csv'
fn='/home/ns/wreg/matlab/sawyer_experiments/jps3.csv'

#read csv using numpy
import numpy as np
data=np.genfromtxt(fn,delimiter=',',skip_header=1)[:, 1:8]


data=data[::-1] #weired reason

data.shape

(159, 7)

In [4]:
action = np.zeros(8)
for i in range(5):
    action[-1] = -0.1  #open
    env.step(action)
    env.render()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/ns/anaconda3/envs/robosuite/lib/python3.8/site-packages/cv2/qt/plugins"


In [5]:
# env.reset()
pos=data[0]
env.robots[0].set_robot_joint_positions(pos)
env.render()

In [6]:
for pos in data:
    env.robots[0].set_robot_joint_positions(pos)
    env.render()

    time.sleep(0.03)

In [7]:
action = np.zeros(8)
for i in range(5):
    action[-1] = 0.1  #close
    env.step(action)
    env.render()

In [8]:
for pos in data[::-1]:
    env.robots[0].set_robot_joint_positions(pos)
    env.render()

    time.sleep(0.03)

In [177]:
# env.robots[0].current_joint_positions()

In [184]:
def vel_gotopos(target, kp=0.3):
    cpos=env.sim.data.qpos[:7]
    dpos=target-cpos
    while np.linalg.norm(dpos)>0.1:
        cpos=env.sim.data.qpos[:7]
        dpos=target-cpos
        a=dpos*kp
        action=np.zeros(8)
        action[:7]=a
        env.step(action)
        env.render()
        # time.sleep(0.03)


def gripper_close(n=15):
    action = np.zeros(8)
    for i in range(n):
        action[-1] = 0.1  #close
        env.step(action)
        env.render()
        
def gripper_open(n=15):
    action = np.zeros(8)
    for i in range(n):
        action[-1] = -0.1  
        env.step(action)
        env.render()

In [57]:
vel_gotopos(data[0], kp=0.5)

In [56]:
vel_gotopos(data[-1], kp=0.5)

In [58]:
env.close()

### modified table

In [70]:
import argparse
import os
from glob import glob

import numpy as np 
import robosuite as suite 
import time

from robosuite.utils.placement_samplers import UniformRandomSampler
from robosuite.utils.mjcf_utils import CustomMaterial
from robosuite.models.objects import BoxObject

In [148]:
table_offset=np.array((0, 0, 1.2))

# initialize objects of interest
tex_attrib = {
    "type": "cube",
}
mat_attrib = {
    "texrepeat": "1 1",
    "specular": "0.4",
    "shininess": "0.1",
}
redwood = CustomMaterial(
    texture="WoodRed",
    tex_name="redwood",
    mat_name="redwood_mat",
    tex_attrib=tex_attrib,
    mat_attrib=mat_attrib,
)


cube = BoxObject(
    name="cube",
    size_min=[0.020, 0.020, 0.020],  # [0.015, 0.015, 0.015],
    size_max=[0.022, 0.022, 0.022],  # [0.018, 0.018, 0.018])
    rgba=[1, 0, 0, 1],
    material=redwood,
)


placement_initializer = UniformRandomSampler(
                name="ObjectSampler",
                mujoco_objects= cube,
                x_range=[-0.02, -0.02],
                y_range=[0.08, 0.08],
                rotation=0,
                ensure_object_boundary_in_range=False,
                ensure_valid_placement=True,
                reference_pos= table_offset,
                z_offset=0.03,
            )

In [149]:
env = suite.make(
    "Lift",
    robots="Sawyer",
    ignore_done=True,
    use_camera_obs=False,
    has_renderer=True,
    has_offscreen_renderer=False,
    control_freq=20, 
    table_offset=np.array((0, 0, 0.96)),
    placement_initializer=placement_initializer,
)

In [150]:
env.reset()
pass 

In [175]:
vel_gotopos(data[0], kp=0.5)

In [174]:
gripper_open()

In [173]:
vel_gotopos(data[-1], kp=0.5)

In [153]:
for pos in data:
    vel_gotopos(pos, kp=0.5)

In [171]:
gripper_close()

In [166]:
gripper_open()

In [164]:
def idle(n=50):
    action = np.zeros(8)
    for i in range(n): 
        env.step(action)
        env.render()

In [165]:
idle()

In [176]:
env.close()